## <u>Notebook by John Uzoma</u>

#### Load JSON file to a flattened pySpark dataframe

In [ ]:
from pyspark.sql.functions import expr

# Create a function to flatten the dataframe and select required columns
def flatten_df(df_name):
   return df_name.select(
            expr("forecasts.regions[0][1].issued").alias("Issued"),
            expr("forecasts.regions[0][2].today").alias("Today"),
            expr("forecasts.regions[0][3].tonight").alias("Tonight"),
            expr("forecasts.regions[0][4].tomorrow").alias("Tomorrow"),
            expr("forecasts.regions[0][5].outlook").alias("Outlook")
        )

# Define the path to the JSON file
json_file_path = "Files/Bronze/DublinTextForecast.json"

# Load JSON data into a dataframe
df = flatten_df(spark.read.json(json_file_path))

#### Replace nulls and blanks in Outlook column

In [ ]:
from pyspark.sql.functions import when, lit, col
 
# Replace null or empty values in Outlook column with "Unknown"
df = df.withColumn("Outlook", when((col("Outlook").isNull() | (col("Outlook")=="")),lit("Unknown")).otherwise(col("Outlook")))

#### More transformations

In [ ]:
from pyspark.sql.functions import substring, to_date, from_utc_timestamp

# Convert Issued column to Irish timestamp
df = df.withColumn("Issued", from_utc_timestamp(col("Issued"), "Europe/Dublin"))

# Add 'Issue_Time' column and convert Issued column to date type
df = df.withColumn("TextForecastIssueTime", substring(col("Issued"), 12, 19)) \
       .withColumn("Issued", to_date(col("Issued")))

#### Define schema for silver table

In [ ]:
from pyspark.sql.types import TimestampType, StringType, DateType
from delta.tables import DeltaTable

DeltaTable.createIfNotExists(spark) \
    .tableName("lakehouse.dublintextforecast_silver") \
    .addColumn("Issued", DateType()) \
    .addColumn("TextForecastIssueTime", StringType()) \
    .addColumn("Today", StringType()) \
    .addColumn("Tonight", StringType()) \
    .addColumn("Tomorrow", StringType()) \
    .addColumn("Outlook", StringType()) \
    .execute()

#### Optimize Delta table writes

In [ ]:
 # Enable V-Order
 spark.conf.set("spark.sql.parquet.vorder.enabled", "true")
    
 # Enable automatic Delta optimized write
 spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")

#### Write dataframe to the silver table (overwrite operation)

In [ ]:
df.write.format("delta").mode("overwrite").save("Tables/dublintextforecast_silver")